In [64]:
import pandas as pd
from openpyxl import load_workbook
import glob
from pathlib import Path
pd.set_option('display.expand_frame_repr', False)  # Evita que las filas se dividan
pd.set_option('display.max_columns', None)  # Muestra todas las columnas


In [65]:
wb = load_workbook('config.xlsx')
ws = wb.active
valor = ws.cell(row=3, column=2).value

carpeta = Path(valor) 
archivo = list(carpeta.rglob("CRM*.xlsx")) + list(carpeta.glob("CRM*.xls")) # se filtra por CRM xlsx u xls
archivos_str = [str(a) for a in archivo] # retorna un array ['xxx','xxxx',...]de las ruta de los archivos

In [ ]:
def leer_excel_smart(path, hoja_principal="Base CRM", hoja_backup="Hoja1"):
    """
    Lee un Excel detectando:
    - qué hoja usar (principal o backup)
    - primera fila válida como encabezado
    - limpia filas vacías
    - estandariza nombres de columnas
    """

    column_map = {
        "No.": "numero",
        "No": "numero",
        "N°": "numero",
        "nro": "numero",

        "Fecha": "fecha",
        "Asesor": "asesor",
        "Contacto": "contacto",
        "Cargo": "cargo",

        "teléfono": "telefono",
        "telefono": "telefono",
        "Teléfono": "telefono",

        "Mail": "mail",
        "Correo": "mail",

        "Razón Social": "razon_social",
        "Razon Social": "razon_social",

        "RUC": "ruc",
        "Web": "web",
        "Departamento": "departamento",
        "Distrito": "distrito",
        "Rubro": "rubro",
        "Tipo de producto": "tipo_producto",
        "Tipo de Producto": "tipo_producto",

        "Origen del lead": "origen_lead",
        "Descripción": "descripcion",
        "Estado": "estado",
        "Monto de la oportunidad": "monto_oportunidad",
        "No. Cotización": "numero_cotizacion",

        "Valor Cotización": "valor_cotizacion",
        "Valor Cotización + IGV": "valor_cotizacion",

        "Fecha seguimiento": "fecha_seguimiento",
        "Semana": "semana",
    }

    try:
        xls = pd.ExcelFile(path)
    except Exception as e:
        print(f"❌ Error al abrir archivo {path}: {e}")
        return None

    if hoja_principal in xls.sheet_names:
        hoja = hoja_principal
    elif hoja_backup in xls.sheet_names:
        hoja = hoja_backup
    else:
        print(f"❌ Ninguna hoja válida encontrada en: {path}")
        return None

    df_raw = pd.read_excel(path, sheet_name=hoja, header=None)

    header_row = next((i for i in range(len(df_raw)) if df_raw.iloc[i].notna().any()), None) # busca la primera fila que no esta vacia

    if header_row is None:
        print(f"❌ No se encontró encabezado en: {path}")
        return None

    df = pd.read_excel(path, sheet_name=hoja, header=header_row)

    df = df.dropna(how='all').reset_index(drop=True) # quita las fila completamente vacias

    # Estandarizar columnas
    df.columns = df.columns.astype(str).str.strip()  # quita espacios
    
    df = df.rename(columns=lambda c: column_map.get(c, c.lower().replace(" ", "_")))
    
    return df



In [ ]:
# Cierre



In [ ]:
archivo_lectura = []
archivo_cierre = []
for archivo  in archivos_str:
    df_1 = leer_excel_smart(archivo, hoja_principal="Base CRM", hoja_backup="CRM")
    if df_1 is None:
        print(f"Saltando archivo no válido: {archivo}")
        continue
    archivo_lectura.append(df_1)
    
    

In [8]:
df_fina = pd.concat(archivo_lectura, ignore_index=True) # creamos un dataframe 

In [ ]:
df_fina.head(10)

In [21]:
df_fina.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   numero             452 non-null    int64         
 1   fecha              452 non-null    datetime64[ns]
 2   asesor             452 non-null    object        
 3   contacto           366 non-null    object        
 4   cargo              314 non-null    object        
 5   telefono           417 non-null    object        
 6   mail               332 non-null    object        
 7   razon_social       432 non-null    object        
 8   ruc                244 non-null    object        
 9   web                45 non-null     object        
 10  departamento       395 non-null    object        
 11  distrito           325 non-null    object        
 12  rubro              421 non-null    object        
 13  tipo_producto      99 non-null     object        
 14  origen_lea

In [34]:
with pd.ExcelWriter("salida.xlsx", engine="openpyxl") as writer:
    df_fina.to_excel(writer, index=False)
    wb = writer.book
    ws = writer.sheets['Sheet1']
    
    for col in ['B','U']:
        for cell in ws[col]:
            if cell.value is not None:
                cell.number_format = "DD/MM/YYYY"
            
    for col in ['D','E','F']:
        for cell in ws[col]:
            if cell.value is not None:
                cell.number_format = "@"
              
    

In [57]:
# desarrollo de cierre
claves = ['No.','Fecha','Asesor','Razon Social','RUC']
pattern = "|".join(claves)

dfCierre = pd.read_excel('CRM Gerencia.xlsx', sheet_name='CIERRES', header=None)

# Buscar fila donde están los encabezados
filaCierre = dfCierre.index[
    dfCierre.apply(lambda row: row.astype(str).str.contains(pattern, case=False).any(), axis=1)
][0]

print("Fila encontrada:", filaCierre)

# Cargar excel usando esa fila como encabezado
df_final = pd.read_excel(
    'CRM Gerencia.xlsx',
    sheet_name='CIERRES',
    header=filaCierre
)

print(df_final)

Fila encontrada: 2
   No.      fecha  Asesor               Razon Social           RUC  Código                                    Producto       Modelo             KG Marca  Costos     Sub Total          Igv      Total
0    1 2025-11-13     NaN    Quiñones Alvarez Angelo           NaN     NaN                            Centro de Lavado  Giant C Max     10 a 13 KG    LG     NaN   3474.580000   625.424000   4100.004
1    2 2025-11-15     NaN        Seventh Heaven SCRL  2.060165e+10     NaN  Rodillo de planchado industrial automatico       G14.25  1450 x 270 mm   GMP     NaN  14406.779661  2593.220339  17000.000
2    3 2025-11-19     NaN  Corporacion Munakuyki SAC  2.061362e+10     NaN         Lavadora centrifuga semi industrial    Titan Max          17 KG    LG     NaN   3950.000000   711.000000   4661.000


In [69]:
from sqlalchemy import true


datosCierre = []
for archivo in archivos_str:
    # desarrollo de cierre
    claves = ['No.','Fecha','Asesor','Razon Social','RUC']
    pattern = "|".join(claves)
    dfCierre = pd.read_excel(archivo, sheet_name='CIERRE', header=None)

    # Buscar fila donde están los encabezados
    filaCierre = dfCierre.index[
        dfCierre.apply(lambda row: row.astype(str).str.contains(pattern, case=False).any(), axis=1)
    ][0]

    print("Fila encontrada:", filaCierre)

    # Cargar excel usando esa fila como encabezado
    df_final = pd.read_excel(
        archivo,
        sheet_name='CIERRE',
        header=filaCierre
    )

    datosCierre.append(df_final)
    
dfCierres = pd.concat(datosCierre, ignore_index=True)
dfCierres

Fila encontrada: 2
Fila encontrada: 2
Fila encontrada: 2


,No.,Fecha,Asesor,Razon Social,RUC,Código,Producto,Modelo,KG,Marca,Costo,Sub Total,Igv,Total,Costos,fecha
0,1,2025-11-13,NaN,Quiñones Alvarez Angelo,NaN,NaN,Centro de Lavado,Giant C Max,10 a 13 KG,LG,NaN,3474.580000,625.424000,4100.004,NaN,NaT
1,2,2025-11-15,NaN,Seventh Heaven SCRL,2.060165e+10,NaN,Rodillo de planchado industrial automatico,G14.25,1450 x 270 mm,GMP,NaN,14406.779661,2593.220339,17000.000,NaN,NaT
2,3,2025-11-19,NaN,Corporacion Munakuyki SAC,2.061362e+10,NaN,Lavadora centrifuga semi industrial,Titan Max,17 KG,LG,NaN,3950.000000,711.000000,4661.000,NaN,NaT
3,1,2025-11-13,NaN,Quiñones Alvarez Angelo,NaN,NaN,Centro de Lavado,Giant C Max,10 a 13 KG,LG,NaN,3474.580000,625.424000,4100.004,NaN,NaT
4,2,2025-11-15,NaN,Seventh Heaven SCRL,2.060165e+10,NaN,Rodillo de planchado industrial automatico,G14.25,1450 x 270 mm,GMP,NaN,14406.779661,2593.220339,17000.000,NaN,NaT
5,3,2025-11-19,NaN,Corporacion Munakuyki SAC,2.061362e+10,NaN,Lavadora centrifuga semi industrial,Titan Max,17 KG,LG,NaN,3950.000000,711.000000,4661.000,NaN,NaT
6,1,NaT,NaN,Quiñones Alvarez Angelo,NaN,NaN,Centro de Lavado,Giant C Max,10 a 13 KG,LG,NaN,3474.580000,625.424000,4100.004,NaN,2025-11-13
7,2,NaT,NaN,Seventh Heaven SCRL,2.060165e+10,NaN,Rodillo de planchado industrial automatico,G14.25,1450 x 270 mm,GMP,NaN,14406.779661,2593.220339,17000.000,NaN,2025-11-15
8,3,NaT,NaN,Corporacion Munakuyki SAC,2.061362e+10,NaN,Lavadora centrifuga semi industrial,Titan Max,17 KG,LG,NaN,3950.000000,711.000000,4661.000,NaN,2025-11-19
